In [80]:
example = open('example.txt', 'r').read()
puzzle = open('puzzle.txt', 'r').read()
test = open('test.txt', 'r').read()

input = puzzle

# Part 1

In [81]:
class Disk():
    def __init__(self, disk_map):
        # Initialise the disk
        self.disk = []

        file=True
        cur_id = 0

        for digit in disk_map:
            # Append this digit to the disk and increment ID
            if file:
                for _ in range(int(digit)):
                    self.disk.append(str(cur_id))
                cur_id += 1
            else:
                for _ in range(int(digit)):
                    self.disk.append('.')
            
            # Alternate between a file and free space
            file = not file
            
    def print_disk(self):
        out_str = ''
        print(self.disk)
        for s in self.disk:
            out_str += s
        
        
        print(out_str)

    def clean_end_space(self):
        '''
        Removes trailing .s from the end of the disk
        '''
        i = -1
        while(self.disk[-1] == '.'):
            del self.disk[-1]

    def move_block(self):
        '''
        Moves the rightmost file block of the disk to the first free slot on the left
        '''
        # Get rightmost block

        # Clear trailing .s
        self.clean_end_space()

        rightmost_block = self.disk[-1]

        # Get first free block
        # Find the first occurrence of a '.' - remember we cleaned the end space of trailing .s so there's definitely a block somewhere after this
        i = 0
        while(self.disk[i] != '.'):
            i += 1
            if i == len(self.disk):
                return False
        
        first_free_index = i

        # Move the block
        self.disk[first_free_index] = rightmost_block
        del self.disk[-1]

        return True

        
    def keep_moving_blocks(self):
        '''
        Iteratively calls move_block until we can't anymore
        '''
        i = 0
        while(self.move_block()):
            #self.print_disk()
            i += 1
            if i > 50000:
                break
    
    def find_checksum(self):
        checksum = 0
        for i in range(len(self.disk)):
            checksum += i*int(self.disk[i])
        return checksum


In [82]:
disk_map = input

disk = Disk(disk_map)

#disk.print_disk()
disk.keep_moving_blocks()
#disk.print_disk()
disk.find_checksum()

# A bit inefficient, but hey - it works!

6430446922192

# Part 2

In [83]:
class Disk():
    def __init__(self, disk_map):
        # Initialise the disk
        self.disk = []

        file=True
        cur_id = 0

        # Part 2 additions - store the lengths of each id block to help us later...
        self.id_lengths = {}
        # ...as well as the ids that we have tried to move thus far
        self.already_tried_to_move = set()

        for digit in disk_map:
            # Append this digit to the disk and increment ID
            if file:
                for _ in range(int(digit)):
                    self.disk.append(str(cur_id))
                    self.id_lengths[str(cur_id)] = int(digit)
                cur_id += 1
            else:
                for _ in range(int(digit)):
                    self.disk.append('.')
            
            # Alternate between a file and free space
            file = not file
            
    def print_disk(self):
        out_str = ''
        #print(self.disk)
        for s in self.disk:
            out_str += s
        
        
        print(out_str)

    def clean_end_space(self):
        '''
        Removes trailing .s from the end of the disk
        '''
        while(self.disk[-1] == '.'):
            del self.disk[-1]

    def move_block(self):
        '''
        Moves the rightmost file block of the disk to the first free slot on the left
        '''

        # Clear trailing .s
        self.clean_end_space()

        # Right pointer determines what block is going to be moved next from right to left, and starts at the right end of the disk
        # If it reaches the left end of the disk, we couldn't find one and so can't move anything (so algorithm ends/no valid moves)
        right_pointer = len(self.disk) - 1

        # Left pointer similarly tracks the free space blocks starting from the left side and going to the right
        # Exhausting left pointer to the right side of the disk makes right pointer shift down to the next right-most id block
        left_pointer = 0

        while(right_pointer >= 0):
            #self.print_disk()

            # If we moved the right pointer past a block on the right and into a free memory region, keep stepping left until we get to the next block...
            while(self.disk[right_pointer] == '.'):
                right_pointer -= 1
                # ...but if we reach the left side of the disk, we've run out of blocks, so there are no valid moves left - return False to indicate this
                if right_pointer < 0:
                    return False

            # Get rightmost block id and length
            rightmost_block = self.disk[right_pointer]
            rightmost_block_length = self.id_lengths[rightmost_block]

            #print(f'looking at block {rightmost_block}')

            # Now need to find the first free memory slot from the left that the rightmost block can fit into (if it even exists)
            while(left_pointer < len(self.disk) and left_pointer < right_pointer):

                # Skip blocks that we have already tried to move thus far (but still shift the index of pointers etc)
                if rightmost_block in self.already_tried_to_move:
                    break

                # Increment left pointer until we find a .
                if self.disk[left_pointer] != '.':
                    left_pointer += 1
                    continue
                
                # At this point, we've found a memory block, but we need to check if there's enough space for the rightmost block
                # Find the length of this current memory block
                memory_block_length = 0
                while(self.disk[left_pointer+memory_block_length] == '.'):
                    memory_block_length += 1
                
                # Can the right block fit?
                #print(f'can {rightmost_block} fit into index {left_pointer}?')
                if memory_block_length >= rightmost_block_length:
                    # We have a valid move! Move the rightmost block into this spot:
                    #print(f'moving id {rightmost_block} into index {left_pointer}')
                    for i in range(rightmost_block_length):
                        # Place the rightmost block into the free slot...
                        self.disk[left_pointer+i] = rightmost_block
                        # ...and remove its block from where it originally was
                        self.disk[right_pointer-i] = '.'
                    return True
                else:
                    # The current block is too large - skip this block
                    left_pointer += memory_block_length
            
            # Exhausted the left pointer for this block, move the right pointer down to the next block and reset left pointer
            left_pointer = 0
            right_pointer -= rightmost_block_length
            self.already_tried_to_move.add(rightmost_block)

        
    def keep_moving_blocks(self):
        '''
        Iteratively calls move_block until we can't anymore
        '''
        i = 0
        while(self.move_block()):
            #self.print_disk()
            i += 1
            if i > 50000:
                break
    
    def find_checksum(self):
        checksum = 0
        for i in range(len(self.disk)):
            if self.disk[i] != '.':
                checksum += i*int(self.disk[i])
        return checksum


In [85]:
disk_map = input

disk = Disk(disk_map)

#print('input', end=' ')
#disk.print_disk()
disk.keep_moving_blocks()
#print('output', end=' ')
#disk.print_disk()
disk.find_checksum()


6460170593016